## Equity analysis, comparing all models for one store placement


In [9]:
# Import libraries
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import haversine as hs
import gurobipy as gp
from gurobipy import GRB
from shapely import wkt

# Helper modules
import helper_population_allocation as pa
import helper_distance_calculation as dc

# Avoid printing set copy warnings
import warnings
warnings.filterwarnings("ignore")

In [10]:
#Importing files
usda = pd.read_csv('../processed_data/Food_Deserts_Pittburgh.csv')
equity = pd.read_csv('../processed_data/Demographics_Combined_Updated.csv')

In [11]:
usda.columns

Index(['OBJECTID', 'Allegheny_Tracts_FID_1', 'Allegheny_Tracts_STATEFP',
       'Allegheny_Tracts_COUNTYFP', 'Allegheny_Tracts_TRACTCE',
       'Allegheny_Tracts_AFFGEOID', 'Allegheny_Tracts_GEOID',
       'Allegheny_Tracts_NAME', 'Allegheny_Tracts_LSAD',
       'Allegheny_Tracts_ALAND',
       ...
       'USDA_Data_TractBlack', 'USDA_Data_TractAsian', 'USDA_Data_TractNHOPI',
       'USDA_Data_TractAIAN', 'USDA_Data_TractOMultir',
       'USDA_Data_TractHispanic', 'USDA_Data_TractHUNV', 'USDA_Data_TractSNAP',
       'Shape_Length', 'Shape_Area'],
      dtype='object', length=166)

In [12]:
usda["Allegheny_Tracts_GEOID"].dtypes

dtype('int64')

In [13]:
equity.dtypes

CensusTract      int64
Senior           int64
SNAP             int64
NoVehicle        int64
Disability     float64
Race           float64
SingleHH       float64
dtype: object

In [14]:
#rename column 
usda.rename(columns = {'Allegheny_Tracts_GEOID':'GEOID'}, inplace = True)
equity.rename(columns = {'CensusTract':'GEOID'}, inplace = True)
usda

,OBJECTID,Allegheny_Tracts_FID_1,Allegheny_Tracts_STATEFP,Allegheny_Tracts_COUNTYFP,Allegheny_Tracts_TRACTCE,Allegheny_Tracts_AFFGEOID,GEOID,Allegheny_Tracts_NAME,Allegheny_Tracts_LSAD,Allegheny_Tracts_ALAND,...,USDA_Data_TractBlack,USDA_Data_TractAsian,USDA_Data_TractNHOPI,USDA_Data_TractAIAN,USDA_Data_TractOMultir,USDA_Data_TractHispanic,USDA_Data_TractHUNV,USDA_Data_TractSNAP,Shape_Length,Shape_Area
0,1,1,42,3,50900,1400000US42003050900,42003050900,509.0,CT,452643,...,1106.0,3.0,0.0,8.0,37.0,19.0,650.0,460.0,0.045048,0.000046
1,2,2,42,3,70300,1400000US42003070300,42003070300,703.0,CT,428546,...,127.0,257.0,0.0,1.0,49.0,64.0,146.0,19.0,0.028484,0.000046
2,3,3,42,3,120700,1400000US42003120700,42003120700,1207.0,CT,514927,...,782.0,1.0,0.0,3.0,13.0,7.0,127.0,155.0,0.031372,0.000055
3,4,4,42,3,140400,1400000US42003140400,42003140400,1404.0,CT,834995,...,55.0,95.0,0.0,8.0,54.0,62.0,50.0,11.0,0.038374,0.000089
4,5,5,42,3,180700,1400000US42003180700,42003180700,1807.0,CT,393380,...,391.0,7.0,0.0,5.0,71.0,43.0,172.0,222.0,0.026142,0.000042
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,174,396,42,3,111300,1400000US42003111300,42003111300,1113.0,CT,536950,...,1795.0,51.0,11.0,6.0,117.0,65.0,405.0,300.0,0.039902,0.000057
174,175,397,42,3,130300,1400000US42003130300,42003130300,1303.0,CT,546310,...,1083.0,2.0,0.0,5.0,38.0,12.0,369.0,313.0,0.037516,0.000057
175,176,398,42,3,141000,1400000US42003141000,42003141000,1410.0,CT,507878,...,17.0,29.0,1.0,1.0,23.0,23.0,22.0,4.0,0.046762,0.000053
176,177,399,42,3,202300,1400000US42003202300,42003202300,2023.0,CT,2532255,...,169.0,211.0,0.0,2.0,111.0,56.0,108.0,97.0,0.076030,0.000269


In [15]:
#merge files
merged = pd.merge(usda, equity, on="GEOID")
merged

,OBJECTID,Allegheny_Tracts_FID_1,Allegheny_Tracts_STATEFP,Allegheny_Tracts_COUNTYFP,Allegheny_Tracts_TRACTCE,Allegheny_Tracts_AFFGEOID,GEOID,Allegheny_Tracts_NAME,Allegheny_Tracts_LSAD,Allegheny_Tracts_ALAND,...,USDA_Data_TractHUNV,USDA_Data_TractSNAP,Shape_Length,Shape_Area,Senior,SNAP,NoVehicle,Disability,Race,SingleHH
0,1,1,42,3,50900,1400000US42003050900,42003050900,509.0,CT,452643,...,650.0,460.0,0.045048,0.000046,179,460,650,264.0,92.7,184.0
1,2,2,42,3,70300,1400000US42003070300,42003070300,703.0,CT,428546,...,146.0,19.0,0.028484,0.000046,325,19,146,153.0,3.4,75.0
2,3,3,42,3,120700,1400000US42003120700,42003120700,1207.0,CT,514927,...,127.0,155.0,0.031372,0.000055,201,155,127,NaN,NaN,NaN
3,4,4,42,3,140400,1400000US42003140400,42003140400,1404.0,CT,834995,...,50.0,11.0,0.038374,0.000089,360,11,50,111.0,1.7,62.0
4,5,5,42,3,180700,1400000US42003180700,42003180700,1807.0,CT,393380,...,172.0,222.0,0.026142,0.000042,200,222,172,280.0,28.0,115.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,174,396,42,3,111300,1400000US42003111300,42003111300,1113.0,CT,536950,...,405.0,300.0,0.039902,0.000057,346,300,405,312.0,31.0,53.0
172,175,397,42,3,130300,1400000US42003130300,42003130300,1303.0,CT,546310,...,369.0,313.0,0.037516,0.000057,285,313,369,NaN,NaN,NaN
173,176,398,42,3,141000,1400000US42003141000,42003141000,1410.0,CT,507878,...,22.0,4.0,0.046762,0.000053,136,4,22,NaN,NaN,NaN
174,177,399,42,3,202300,1400000US42003202300,42003202300,2023.0,CT,2532255,...,108.0,97.0,0.076030,0.000269,806,97,108,308.0,14.3,27.0


In [16]:
merged["USDA_Data_PovertyRate"]

0      58.4
1      19.2
2      43.4
3       5.8
4      19.0
       ... 
171    17.8
172    35.4
173     1.2
174     9.5
175    10.1
Name: USDA_Data_PovertyRate, Length: 176, dtype: float64

In [17]:
#Add additional columns 
merged["PctSeniorPop"] = round((merged["Senior"] / merged["USDA_Data_Pop2010"])*100,2)  
merged["PctSNAPHH"] = round((merged["SNAP"] / merged["USDA_Data_OHU2010"])*100, 2)
merged["PctSingleHH"] = round((merged["SingleHH"] / merged["USDA_Data_OHU2010"])*100, 2)
merged["PctNoVehicleHH"] = round((merged["NoVehicle"] / merged["USDA_Data_OHU2010"])*100, 2)
merged["PctDisabilityPop"] = round((merged["Disability"] / merged["USDA_Data_Pop2010"])*100, 2)
merged["Race"] = merged["Race"]/100
merged["Black_Pop"] = round(merged["Race"]*merged["USDA_Data_Pop2010"],0)
merged["USDA_Data_PovertyRate"] = merged["USDA_Data_PovertyRate"] / 100

In [18]:
merged

,OBJECTID,Allegheny_Tracts_FID_1,Allegheny_Tracts_STATEFP,Allegheny_Tracts_COUNTYFP,Allegheny_Tracts_TRACTCE,Allegheny_Tracts_AFFGEOID,GEOID,Allegheny_Tracts_NAME,Allegheny_Tracts_LSAD,Allegheny_Tracts_ALAND,...,NoVehicle,Disability,Race,SingleHH,PctSeniorPop,PctSNAPHH,PctSingleHH,PctNoVehicleHH,PctDisabilityPop,Black_Pop
0,1,1,42,3,50900,1400000US42003050900,42003050900,509.0,CT,452643,...,650,264.0,0.927,184.0,14.89,70.66,28.26,99.85,21.96,1114.0
1,2,2,42,3,70300,1400000US42003070300,42003070300,703.0,CT,428546,...,146,153.0,0.034,75.0,14.79,1.61,6.37,12.39,6.96,75.0
2,3,3,42,3,120700,1400000US42003120700,42003120700,1207.0,CT,514927,...,127,NaN,NaN,NaN,24.57,37.44,NaN,30.68,NaN,NaN
3,4,4,42,3,140400,1400000US42003140400,42003140400,1404.0,CT,834995,...,50,111.0,0.017,62.0,15.61,1.16,6.54,5.27,4.81,39.0
4,5,5,42,3,180700,1400000US42003180700,42003180700,1807.0,CT,393380,...,172,280.0,0.280,115.0,9.95,24.67,12.78,19.11,13.92,563.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,174,396,42,3,111300,1400000US42003111300,42003111300,1113.0,CT,536950,...,405,312.0,0.310,53.0,13.11,24.25,4.28,32.74,11.82,818.0
172,175,397,42,3,130300,1400000US42003130300,42003130300,1303.0,CT,546310,...,369,NaN,NaN,NaN,24.72,52.87,NaN,62.33,NaN,NaN
173,176,398,42,3,141000,1400000US42003141000,42003141000,1410.0,CT,507878,...,22,NaN,NaN,NaN,14.66,0.88,NaN,4.82,NaN,NaN
174,177,399,42,3,202300,1400000US42003202300,42003202300,2023.0,CT,2532255,...,108,308.0,0.143,27.0,19.45,4.80,1.34,5.34,7.43,593.0


## Model 1

In [19]:
# Get the main buildings dataset 
buildings_df = gpd.read_file('../processed_data/relevant_buildings.shp')

# Create ID variable
buildings_df.reset_index(drop=True, inplace=True)
buildings_df['building_id'] = buildings_df.index + 1
buildings_df['building_id'] = buildings_df.apply(lambda row: str(row['building_id']) + '-' + str(row['CLASS']) , axis=1)

In [20]:
# Matching GEOIDs

acs_data = pd.read_csv('ACSData.csv')
acs_data = acs_data[['geometry', 'geo_id','B01003_001E', 'B25010_001E']]
acs_data['geometry'] = acs_data['geometry'].apply(wkt.loads)
acs_data = gpd.GeoDataFrame(acs_data, geometry = 'geometry').set_crs(buildings_df.crs)

buildings_df = buildings_df.sjoin(acs_data, how="left", predicate = 'intersects')
buildings_df["geo_id"] = buildings_df["geo_id"]
buildings_df = buildings_df.drop_duplicates("building_id")

In [21]:
buildings_df

,CLASS,class_reco,hood,geoid10,tractce10,geometry,building_id,index_right,geo_id,B01003_001E,B25010_001E
0,C,1-Unit Residential,Brookline,420031918003,191800,"POLYGON ((-80.01230 40.38309, -80.01255 40.383...",1-C,248,1400000US42003479000,2103.0,2.43
1,R,1-Unit Residential,Brookline,420031918003,191800,"POLYGON ((-80.01382 40.38638, -80.01380 40.386...",2-R,248,1400000US42003479000,2103.0,2.43
2,R,1-Unit Residential,Brookline,420031918003,191800,"POLYGON ((-80.01309 40.38253, -80.01307 40.382...",3-R,248,1400000US42003479000,2103.0,2.43
3,R,1-Unit Residential,Brookline,420031918003,191800,"POLYGON ((-80.01399 40.38554, -80.01399 40.385...",4-R,248,1400000US42003479000,2103.0,2.43
4,R,1-Unit Residential,Brookline,420031918003,191800,"POLYGON ((-80.00736 40.38204, -80.00729 40.381...",5-R,248,1400000US42003479000,2103.0,2.43
...,...,...,...,...,...,...,...,...,...,...,...
116273,C,commercial,Point Breeze,420039811001,981100,"POLYGON ((-79.90934 40.44247, -79.90928 40.442...",116274-C,390,1400000US42003981100,0.0,NaN
116274,R,2-Unit Residential,Squirrel Hill South,420039803001,980300,"POLYGON ((-79.91447 40.43197, -79.91443 40.431...",116275-R,51,1400000US42003140800,4374.0,2.26
116275,C,commercial,Squirrel Hill South,420039803001,980300,"POLYGON ((-79.90925 40.42686, -79.90910 40.426...",116276-C,51,1400000US42003140800,4374.0,2.26
116276,R,1-Unit Residential,Squirrel Hill South,420039803001,980300,"POLYGON ((-79.91190 40.43303, -79.91190 40.433...",116277-R,51,1400000US42003140800,4374.0,2.26


In [22]:
# Create arrays to track ordering (residential)
res_buildings = buildings_df[buildings_df['class_reco'].str.contains('Residential')]
res_buildings = res_buildings.sort_values('building_id')
res_buildings = dc.get_geocoordinate(res_buildings, 'geometry')

res_buildings_array = np.array(res_buildings['building_id'])
res_buildings_coordinates_array = np.array(res_buildings['coordinates'])

In [23]:
res_buildings

,CLASS,class_reco,hood,geoid10,tractce10,geometry,building_id,index_right,geo_id,B01003_001E,B25010_001E,centroids,coordinates
0,C,1-Unit Residential,Brookline,420031918003,191800,"POLYGON ((-1280260.228 6136598.189, -1280260.0...",1-C,248,1400000US42003479000,2103.0,2.43,POINT (-80.01241 40.38317),"(-80.01241198145034, 40.383174075305014)"
9,R,1-Unit Residential,Brookline,420031918003,191800,"POLYGON ((-1280199.622 6136721.733, -1280189.9...",10-R,71,1400000US42003191800,5032.0,2.16,POINT (-80.01329 40.38389),"(-80.01328831494197, 40.383892741379555)"
99,R,1-Unit Residential,Brookline,420031918003,191800,"POLYGON ((-1279976.335 6136247.148, -1279971.8...",100-R,71,1400000US42003191800,5032.0,2.16,POINT (-80.00693 40.38506),"(-80.00693119400655, 40.38505578689698)"
999,R,1-Unit Residential,Brookline,420031918003,191800,"POLYGON ((-1279599.432 6137959.684, -1279602.7...",1000-R,71,1400000US42003191800,5032.0,2.16,POINT (-80.02354 40.39119),"(-80.0235433223972, 40.39119376093412)"
9999,R,1-Unit Residential,Overbrook,420033207001,320700,"POLYGON ((-1280279.261 6135210.671, -1280267.3...",10000-R,97,1400000US42003320700,1710.0,2.31,POINT (-79.99736 40.38076),"(-79.99735746686683, 40.38075854377677)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99994,R,1-Unit Residential,Squirrel Hill South,420031408003,140800,"POLYGON ((-1273561.660 6131008.563, -1273569.9...",99995-R,51,1400000US42003140800,4374.0,2.26,POINT (-79.91616 40.43421),"(-79.9161613197582, 40.43421429599738)"
99995,R,2-Unit Residential,Squirrel Hill South,420031408003,140800,"POLYGON ((-1273670.799 6131133.428, -1273668.5...",99996-R,51,1400000US42003140800,4374.0,2.26,POINT (-79.91799 40.43346),"(-79.91798881834804, 40.43345644852324)"
99996,R,1-Unit Residential,Squirrel Hill South,420031408003,140800,"POLYGON ((-1273343.856 6130758.732, -1273342.4...",99997-R,51,1400000US42003140800,4374.0,2.26,POINT (-79.91214 40.43576),"(-79.91214182299784, 40.43576063450121)"
99997,R,2-Unit Residential,Squirrel Hill South,420031408003,140800,"POLYGON ((-1273929.466 6130854.863, -1273928.4...",99998-R,51,1400000US42003140800,4374.0,2.26,POINT (-79.91636 40.43066),"(-79.91635744600651, 40.43066344323655)"


In [24]:
# Create arrays to track ordering (Commercial)
comm_buildings = buildings_df[buildings_df['class_reco'].str.contains('commercial')]
comm_buildings = comm_buildings.sort_values('building_id')
comm_buildings = dc.get_geocoordinate(comm_buildings, 'geometry')

comm_buildings_array = np.array(comm_buildings['building_id'])
comm_buildings_coordinates_array = np.array(comm_buildings['coordinates'])


In [25]:
# Create arrays to track ordering (grocery stores)
grocery_stores = buildings_df[buildings_df['class_reco'].str.contains('Grocery')]
grocery_stores = grocery_stores.sort_values('building_id')
grocery_stores = dc.get_geocoordinate(grocery_stores, 'geometry')

grocery_stores_array = np.array(grocery_stores['building_id'])
grocery_stores_coordinates_array = np.array(grocery_stores['coordinates'])


In [26]:
#New Stores
chosen_stores = pd.read_csv('new_store_ids.csv')
chosen_stores

chosen_store = chosen_stores['building_id'].iloc[0]
chosen_store

'31851-C'

In [27]:
# Importing Distance Matrix
res_comm_distance_matrix = np.load('../processed_data/res_comm_distance_matrix.npy')

# Creating a modified res comm access matrix 
res_comm_access_matrix_half_mile = res_comm_distance_matrix.copy()
res_comm_access_matrix_half_mile[res_comm_access_matrix_half_mile <= 0.5] = 1
res_comm_access_matrix_half_mile[res_comm_access_matrix_half_mile != 1] = 0

In [28]:
res_comm_access_matrix_half_mile

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 1., 0., ..., 1., 0., 0.],
       [1., 1., 1., ..., 1., 0., 1.],
       [1., 1., 1., ..., 1., 0., 1.]])

In [29]:
res_comm_access_matrix_half_mile.shape

(109324, 6895)

In [30]:
%%time

# Create parameter matrices (Res groc access array - Ai)
# ith value indicates whether the ith residential building has existing access
res_groc_distance_matrix, res_groc_access_matrix = dc.calculate_access(res_buildings_coordinates_array, grocery_stores_coordinates_array)

res_groc_access_matrix_half_mile = res_groc_distance_matrix.copy()
res_groc_access_matrix_half_mile[res_groc_access_matrix_half_mile <= 0.5] = 1
res_groc_access_matrix_half_mile[res_groc_access_matrix_half_mile != 1] = 0

res_access_array_half_mile = np.amax(res_groc_access_matrix_half_mile, 1)


Wall time: 30.7 s


In [31]:
res_access_array_half_mile.shape

(109324,)

In [32]:
existing_access_indices = res_access_array_half_mile.nonzero()[0] # These are indices of residential buildings that currently have access
res_comm_access_matrix_subset = np.delete(res_comm_access_matrix_half_mile, existing_access_indices, axis=0 )

In [33]:
chosen_store_index = np.where(comm_buildings_array == chosen_store)[0][0]
chosen_store_index

2206

In [34]:
#New residential buildings that will have access based on commercial building
new_building_with_access = res_comm_access_matrix_half_mile[:,2206]
new_building_with_access = new_building_with_access.nonzero()[0]
new_building_with_access = [i for i in new_building_with_access if i not in existing_access_indices]


In [35]:
# Get IDs of all residential buildings with access
res_buildings_array[new_building_with_access]

array(['27717-R', '27718-R', '27719-R', ..., '84028-R', '84039-R',
       '84043-R'], dtype=object)

In [36]:
new_access_buildings = buildings_df[buildings_df.building_id.isin(res_buildings_array[new_building_with_access])]
new_access_buildings

,CLASS,class_reco,hood,geoid10,tractce10,geometry,building_id,index_right,geo_id,B01003_001E,B25010_001E
27716,R,1-Unit Residential,Brighton Heights,420032708003,270800,"POLYGON ((-80.03906 40.49075, -80.03892 40.490...",27717-R,86,1400000US42003270800,2408.0,1.79
27717,R,1-Unit Residential,Brighton Heights,420032708003,270800,"POLYGON ((-80.04114 40.49117, -80.04108 40.491...",27718-R,86,1400000US42003270800,2408.0,1.79
27718,R,1-Unit Residential,Brighton Heights,420032708003,270800,"POLYGON ((-80.04160 40.49078, -80.04155 40.490...",27719-R,86,1400000US42003270800,2408.0,1.79
27720,R,2-Unit Residential,Brighton Heights,420032708003,270800,"POLYGON ((-80.03473 40.48560, -80.03469 40.485...",27721-R,86,1400000US42003270800,2408.0,1.79
27729,R,1-Unit Residential,Brighton Heights,420032708003,270800,"POLYGON ((-80.04090 40.48914, -80.04080 40.489...",27730-R,86,1400000US42003270800,2408.0,1.79
...,...,...,...,...,...,...,...,...,...,...,...
84024,R,1-Unit Residential,Elliott,420035626002,562600,"POLYGON ((-80.04360 40.44778, -80.04353 40.447...",84025-R,359,1400000US42003562600,2624.0,1.98
84025,R,1-Unit Residential,Elliott,420035626002,562600,"POLYGON ((-80.04359 40.44791, -80.04351 40.447...",84026-R,359,1400000US42003562600,2624.0,1.98
84027,R,2-Unit Residential,Elliott,420035626002,562600,"POLYGON ((-80.04264 40.44376, -80.04265 40.443...",84028-R,359,1400000US42003562600,2624.0,1.98
84038,R,1-Unit Residential,Elliott,420035626002,562600,"POLYGON ((-80.04265 40.44777, -80.04262 40.447...",84039-R,359,1400000US42003562600,2624.0,1.98


In [37]:
#Census Tracts with New Access to Gracery Stores
census_tracts_with_new_access = set(new_access_buildings['geo_id'])
census_tracts_with_new_access

{'1400000US42003270100',
 '1400000US42003270300',
 '1400000US42003270800',
 '1400000US42003271600',
 '1400000US42003281400',
 '1400000US42003562500',
 '1400000US42003562600'}

In [38]:
len(census_tracts_with_new_access)

7

In [39]:
original_buildings_without_access = np.where(res_access_array_half_mile==0)[0] #Buildings that dont have access
remaining_buildings_without_access = [i for i in original_buildings_without_access if i not in new_building_with_access]
no_access_buildings = buildings_df[buildings_df.building_id.isin(res_buildings_array[remaining_buildings_without_access])]

census_tracts_without_access = set(no_access_buildings['geo_id'])
census_tracts_without_access

{'1400000US42003040200',
 '1400000US42003040500',
 '1400000US42003040900',
 '1400000US42003050600',
 '1400000US42003051000',
 '1400000US42003060300',
 '1400000US42003060500',
 '1400000US42003070900',
 '1400000US42003080400',
 '1400000US42003080600',
 '1400000US42003080900',
 '1400000US42003090100',
 '1400000US42003090200',
 '1400000US42003100500',
 '1400000US42003101800',
 '1400000US42003101900',
 '1400000US42003130600',
 '1400000US42003130700',
 '1400000US42003130800',
 '1400000US42003140100',
 '1400000US42003140800',
 '1400000US42003141100',
 '1400000US42003141400',
 '1400000US42003151600',
 '1400000US42003151700',
 '1400000US42003161000',
 '1400000US42003191600',
 '1400000US42003191700',
 '1400000US42003191800',
 '1400000US42003202200',
 '1400000US42003260200',
 '1400000US42003261300',
 '1400000US42003270100',
 '1400000US42003270800',
 '1400000US42003281400',
 '1400000US42003281500',
 '1400000US42003290100',
 '1400000US42003300100',
 '1400000US42003310200',
 '1400000US42003320400',


## Calculations Model 1

In [40]:
tracts_new_access = census_tracts_with_new_access
tracts_no_access = census_tracts_without_access

In [41]:
#Equity Calculations 
new_access = merged[merged.Allegheny_Tracts_AFFGEOID.isin(tracts_new_access)]
no_access = merged[merged.Allegheny_Tracts_AFFGEOID.isin(tracts_no_access)]
new_access

,OBJECTID,Allegheny_Tracts_FID_1,Allegheny_Tracts_STATEFP,Allegheny_Tracts_COUNTYFP,Allegheny_Tracts_TRACTCE,Allegheny_Tracts_AFFGEOID,GEOID,Allegheny_Tracts_NAME,Allegheny_Tracts_LSAD,Allegheny_Tracts_ALAND,...,NoVehicle,Disability,Race,SingleHH,PctSeniorPop,PctSNAPHH,PctSingleHH,PctNoVehicleHH,PctDisabilityPop,Black_Pop
6,7,7,42,3,270100,1400000US42003270100,42003270100,2701.0,CT,1039795,...,58,400.0,0.202,240.0,12.84,10.15,21.00,5.07,15.10,535.0
7,8,8,42,3,281400,1400000US42003281400,42003281400,2814.0,CT,1328964,...,326,365.0,0.280,290.0,13.78,44.66,29.23,32.86,15.52,659.0
55,58,112,42,3,270300,1400000US42003270300,42003270300,2703.0,CT,593102,...,138,529.0,0.377,126.0,17.26,25.20,14.24,15.59,27.17,734.0
56,59,113,42,3,270800,1400000US42003270800,42003270800,2708.0,CT,1270794,...,146,224.0,0.158,23.0,19.65,11.94,1.87,11.86,8.45,419.0
88,91,203,42,3,562500,1400000US42003562500,42003562500,5625.0,CT,2029314,...,339,432.0,0.582,352.0,10.39,33.19,29.88,28.78,14.25,1765.0
154,157,350,42,3,562600,1400000US42003562600,42003562600,5626.0,CT,2092793,...,205,526.0,0.279,210.0,12.52,31.54,18.25,17.81,19.96,735.0


In [42]:
no_access 

,OBJECTID,Allegheny_Tracts_FID_1,Allegheny_Tracts_STATEFP,Allegheny_Tracts_COUNTYFP,Allegheny_Tracts_TRACTCE,Allegheny_Tracts_AFFGEOID,GEOID,Allegheny_Tracts_NAME,Allegheny_Tracts_LSAD,Allegheny_Tracts_ALAND,...,NoVehicle,Disability,Race,SingleHH,PctSeniorPop,PctSNAPHH,PctSingleHH,PctNoVehicleHH,PctDisabilityPop,Black_Pop
6,7,7,42,3,270100,1400000US42003270100,42003270100,2701.00,CT,1039795,...,58,400.0,0.202,240.0,12.84,10.15,21.00,5.07,15.10,535.0
7,8,8,42,3,281400,1400000US42003281400,42003281400,2814.00,CT,1328964,...,326,365.0,0.280,290.0,13.78,44.66,29.23,32.86,15.52,659.0
15,16,31,42,3,80600,1400000US42003080600,42003080600,806.00,CT,404921,...,154,126.0,0.013,54.0,8.61,2.70,4.86,13.85,6.31,26.0
20,21,36,42,3,140800,1400000US42003140800,42003140800,1408.00,CT,1089074,...,424,368.0,0.024,12.0,12.21,10.35,0.62,21.73,8.08,109.0
22,23,38,42,3,191800,1400000US42003191800,42003191800,1918.00,CT,1453327,...,302,995.0,0.071,314.0,15.86,14.61,13.07,12.57,18.70,378.0
26,27,56,42,3,482500,1400000US42003482500,42003482500,4825.00,CT,2387778,...,98,350.0,0.127,78.0,24.05,19.54,9.02,11.33,18.14,245.0
31,33,72,42,3,50600,1400000US42003050600,42003050600,506.00,CT,805419,...,265,651.0,0.776,282.0,24.89,16.63,27.76,26.08,31.65,1596.0
32,34,73,42,3,141100,1400000US42003141100,42003141100,1411.00,CT,1226334,...,46,151.0,0.080,11.0,17.93,7.64,1.83,7.64,11.09,109.0
39,41,92,42,3,60500,1400000US42003060500,42003060500,605.00,CT,662686,...,145,228.0,0.049,16.0,16.01,14.06,2.30,20.80,17.90,62.0
41,43,94,42,3,90200,1400000US42003090200,42003090200,902.00,CT,446355,...,364,359.0,0.025,12.0,25.16,9.72,0.88,26.59,13.62,66.0


In [43]:
#Calculations 
#ACCESS 
sum_pop_access = sum(new_access["USDA_Data_Pop2010"]) 
sum_hh_access = sum(new_access["USDA_Data_OHU2010"]) 

#Population
avg_seniors_access = sum(new_access["Senior"])/sum_pop_access 
avg_black_access = sum(new_access["Black_Pop"])/sum_pop_access 
avg_disability_access = sum(new_access["Disability"])/sum_pop_access 

#Households
avg_SNAP_access = sum(new_access["SNAP"]) / sum_hh_access 
avg_single_access = sum(new_access["SingleHH"]) / sum_hh_access
avg_novehicle_access = sum(new_access["NoVehicle"]) / sum_hh_access
avg_poverty_access = sum(new_access["USDA_Data_OHU2010"]*new_access["USDA_Data_PovertyRate"]/sum_hh_access)


In [44]:
print("CENSUS TRACTS WITH ACCESS")
print("Average % Senior Population:", avg_seniors_access)
print("Average % Black Population:", avg_black_access)
print("Average % Disabled Population:", avg_disability_access)

print("Average % Households that are SNAP Recipients:", avg_SNAP_access)
print("Average % Single Head Households:", avg_single_access)
print("Average % Household with No Vehicle:", avg_novehicle_access)
print("Average Poverty Rate:", avg_poverty_access, "\n")



CENSUS TRACTS WITH ACCESS
Average % Senior Population: 0.1418839250622298
Average % Black Population: 0.31750294772697496
Average % Disabled Population: 0.16219048866762742
Average % Households that are SNAP Recipients: 0.25577507598784194
Average % Single Head Households: 0.1886018237082067
Average % Household with No Vehicle: 0.18419452887537993
Average Poverty Rate: 0.2237737082066869 



In [45]:
#NO ACCESS
sum_pop_noaccess = sum(no_access["USDA_Data_Pop2010"]) 
sum_hh_noaccess = sum(no_access["USDA_Data_OHU2010"]) 

#Population
avg_seniors_noaccess = sum(no_access["Senior"])/sum_pop_noaccess
avg_black_noaccess = sum(no_access["Black_Pop"])/sum_pop_noaccess 
avg_disability_noaccess = sum(no_access["Disability"])/sum_pop_noaccess

#Households
avg_SNAP_noaccess = sum(no_access["SNAP"]) / sum_hh_noaccess
avg_single_noaccess = sum(no_access["SingleHH"]) / sum_hh_noaccess
avg_novehicle_noaccess = sum(no_access["NoVehicle"]) / sum_hh_noaccess
avg_poverty_noaccess = sum(no_access["USDA_Data_OHU2010"]*no_access["USDA_Data_PovertyRate"]/sum_hh_noaccess)

In [46]:
print("CENSUS TRACTS WITH NO ACCESS")
print("Average % Senior Population:", avg_seniors_noaccess)
print("Average % Black Population:", avg_black_noaccess)
print("Average % Disabled Population:", avg_disability_noaccess)

print("Average % Households that are SNAP Recipients:", avg_SNAP_noaccess)
print("Average % Single Head Households:", avg_single_noaccess)
print("Average % Household with No Vehicle:", avg_novehicle_noaccess)
print("Average Poverty Rate:", avg_poverty_noaccess, "\n")



CENSUS TRACTS WITH NO ACCESS
Average % Senior Population: 0.16257623523172565
Average % Black Population: 0.1749977902831384
Average % Disabled Population: 0.14374355499248695
Average % Households that are SNAP Recipients: 0.13939981943377666
Average % Single Head Households: 0.09409656250421096
Average % Household with No Vehicle: 0.16556844672622656
Average Poverty Rate: 0.17609987737666924 



## Model 2 

In [47]:
chosen_store ='96061-C'                     #Provided by Mihir

chosen_store_index = np.where(comm_buildings_array == chosen_store)[0][0]
chosen_store_index

6764

In [55]:
#New residential buildings that will have access based on commercial building
new_building_with_access = res_comm_access_matrix_half_mile[:,6764]
new_building_with_access = new_building_with_access.nonzero()[0]
new_building_with_access = [i for i in new_building_with_access if i not in existing_access_indices]

In [56]:
# Get IDs of all residential buildings with access
res_buildings_array[new_building_with_access]

array(['100271-R', '105409-R', '105411-R', ..., '96311-R', '96313-C',
       '96320-R'], dtype=object)

In [57]:
new_access_buildings = buildings_df[buildings_df.building_id.isin(res_buildings_array[new_building_with_access])]
new_access_buildings

,CLASS,class_reco,hood,geoid10,tractce10,geometry,building_id,index_right,geo_id,B01003_001E,B25010_001E
15913,R,1-Unit Residential,Hazelwood,420035629001,562900,"POLYGON ((-79.94658 40.40584, -79.94640 40.405...",15914-R,362,1400000US42003562901,1567.0,2.43
15914,R,1-Unit Residential,Hazelwood,420035629001,562900,"POLYGON ((-79.94595 40.40683, -79.94591 40.406...",15915-R,362,1400000US42003562901,1567.0,2.43
15915,R,1-Unit Residential,Hazelwood,420035629001,562900,"POLYGON ((-79.94037 40.41754, -79.94022 40.417...",15916-R,362,1400000US42003562901,1567.0,2.43
15916,R,1-Unit Residential,Hazelwood,420035629001,562900,"POLYGON ((-79.94529 40.40746, -79.94527 40.407...",15917-R,362,1400000US42003562901,1567.0,2.43
15917,R,1-Unit Residential,Hazelwood,420035629001,562900,"POLYGON ((-79.94226 40.41576, -79.94212 40.415...",15918-R,362,1400000US42003562901,1567.0,2.43
...,...,...,...,...,...,...,...,...,...,...,...
115631,R,1-Unit Residential,Shadyside,420030709002,070900,"POLYGON ((-79.94244 40.44875, -79.94229 40.448...",115632-R,21,1400000US42003070900,4250.0,1.60
115852,R,1-Unit Residential,Shadyside,420030709002,070900,"POLYGON ((-79.94245 40.44887, -79.94235 40.448...",115853-R,21,1400000US42003070900,4250.0,1.60
115949,R,1-Unit Residential,Shadyside,420030709002,070900,"POLYGON ((-79.94226 40.44863, -79.94225 40.448...",115950-R,21,1400000US42003070900,4250.0,1.60
115997,R,1-Unit Residential,Shadyside,420030709002,070900,"POLYGON ((-79.94229 40.44953, -79.94227 40.449...",115998-R,21,1400000US42003070900,4250.0,1.60


In [58]:
#Census Tracts with New Access to Gracery Stores
census_tracts_with_new_access = set(new_access_buildings['geo_id'])
census_tracts_with_new_access

{'1400000US42003070900',
 '1400000US42003140100',
 '1400000US42003151600',
 '1400000US42003151700',
 '1400000US42003562300',
 '1400000US42003562901',
 '1400000US42003565300',
 '1400000US42003980500'}

In [59]:
len(census_tracts_with_new_access)

8

In [60]:
original_buildings_without_access = np.where(res_access_array_half_mile==0)[0] #Buildings that dont have access
remaining_buildings_without_access = [i for i in original_buildings_without_access if i not in new_building_with_access]
no_access_buildings = buildings_df[buildings_df.building_id.isin(res_buildings_array[remaining_buildings_without_access])]

census_tracts_without_access = set(no_access_buildings['geo_id'])
census_tracts_without_access

{'1400000US42003040200',
 '1400000US42003040500',
 '1400000US42003040900',
 '1400000US42003050600',
 '1400000US42003051000',
 '1400000US42003060300',
 '1400000US42003060500',
 '1400000US42003070900',
 '1400000US42003080400',
 '1400000US42003080600',
 '1400000US42003080900',
 '1400000US42003090100',
 '1400000US42003090200',
 '1400000US42003100500',
 '1400000US42003101800',
 '1400000US42003101900',
 '1400000US42003130600',
 '1400000US42003130700',
 '1400000US42003130800',
 '1400000US42003140800',
 '1400000US42003141100',
 '1400000US42003141400',
 '1400000US42003161000',
 '1400000US42003191600',
 '1400000US42003191700',
 '1400000US42003191800',
 '1400000US42003202200',
 '1400000US42003260200',
 '1400000US42003261300',
 '1400000US42003270100',
 '1400000US42003270300',
 '1400000US42003270800',
 '1400000US42003271600',
 '1400000US42003281400',
 '1400000US42003281500',
 '1400000US42003290100',
 '1400000US42003300100',
 '1400000US42003310200',
 '1400000US42003320400',
 '1400000US42003320600',


In [61]:
len(census_tracts_without_access)

63

## Calculations Model 2

In [62]:
tracts_new_access = census_tracts_with_new_access
tracts_no_access = census_tracts_without_access

In [63]:
#Equity Calculations 
new_access = merged[merged.Allegheny_Tracts_AFFGEOID.isin(tracts_new_access)]
no_access = merged[merged.Allegheny_Tracts_AFFGEOID.isin(tracts_no_access)]
new_access

,OBJECTID,Allegheny_Tracts_FID_1,Allegheny_Tracts_STATEFP,Allegheny_Tracts_COUNTYFP,Allegheny_Tracts_TRACTCE,Allegheny_Tracts_AFFGEOID,GEOID,Allegheny_Tracts_NAME,Allegheny_Tracts_LSAD,Allegheny_Tracts_ALAND,...,NoVehicle,Disability,Race,SingleHH,PctSeniorPop,PctSNAPHH,PctSingleHH,PctNoVehicleHH,PctDisabilityPop,Black_Pop
69,72,154,42,3,562300,1400000US42003562300,42003562300,5623.0,CT,3116517,...,573,886.0,0.493,245.0,18.56,43.30,17.11,40.01,25.54,1710.0
81,84,167,42,3,140100,1400000US42003140100,42003140100,1401.0,CT,1465179,...,107,374.0,0.065,0.0,9.13,0.00,0.00,10.40,6.83,356.0
93,96,211,42,3,980500,1400000US42003980500,42003980500,9805.0,CT,1612453,...,0,10.0,0.000,0.0,22.22,0.00,0.00,0.00,37.04,0.0
111,114,276,42,3,70900,1400000US42003070900,42003070900,709.0,CT,803571,...,957,277.0,0.070,96.0,12.16,1.25,3.53,35.20,5.61,345.0
125,128,290,42,3,151600,1400000US42003151600,42003151600,1516.0,CT,820480,...,121,276.0,0.005,33.0,16.25,6.09,2.95,10.83,11.68,12.0
126,129,291,42,3,151700,1400000US42003151700,42003151700,1517.0,CT,1189443,...,372,1192.0,0.091,39.0,14.91,12.92,1.65,15.76,24.17,449.0


In [64]:
#Calculations 
#ACCESS 
sum_pop_access = sum(new_access["USDA_Data_Pop2010"]) 
sum_hh_access = sum(new_access["USDA_Data_OHU2010"]) 

#Population
avg_seniors_access = sum(new_access["Senior"])/sum_pop_access 
avg_black_access = sum(new_access["Black_Pop"])/sum_pop_access 
avg_disability_access = sum(new_access["Disability"])/sum_pop_access 

#Households
avg_SNAP_access = sum(new_access["SNAP"]) / sum_hh_access 
avg_single_access = sum(new_access["SingleHH"]) / sum_hh_access
avg_novehicle_access = sum(new_access["NoVehicle"]) / sum_hh_access
avg_poverty_access = sum(new_access["USDA_Data_OHU2010"]*new_access["USDA_Data_PovertyRate"]/sum_hh_access)


In [65]:
print("CENSUS TRACTS WITH ACCESS")
print("Average % Senior Population:", avg_seniors_access)
print("Average % Black Population:", avg_black_access)
print("Average % Disabled Population:", avg_disability_access)

print("Average % Households that are SNAP Recipients:", avg_SNAP_access)
print("Average % Single Head Households:", avg_single_access)
print("Average % Household with No Vehicle:", avg_novehicle_access)
print("Average Poverty Rate:", avg_poverty_access, "\n")

CENSUS TRACTS WITH ACCESS
Average % Senior Population: 0.1353301886792453
Average % Black Population: 0.13547169811320756
Average % Disabled Population: 0.14221698113207548
Average % Households that are SNAP Recipients: 0.11842712177121771
Average % Single Head Households: 0.047624538745387454
Average % Household with No Vehicle: 0.2456180811808118
Average Poverty Rate: 0.2252512684501845 



In [66]:
#NO ACCESS
sum_pop_noaccess = sum(no_access["USDA_Data_Pop2010"]) 
sum_hh_noaccess = sum(no_access["USDA_Data_OHU2010"]) 

#Population
avg_seniors_noaccess = sum(no_access["Senior"])/sum_pop_noaccess
avg_black_noaccess = sum(no_access["Black_Pop"])/sum_pop_noaccess 
avg_disability_noaccess = sum(no_access["Disability"])/sum_pop_noaccess

#Households
avg_SNAP_noaccess = sum(no_access["SNAP"]) / sum_hh_noaccess
avg_single_noaccess = sum(no_access["SingleHH"]) / sum_hh_noaccess
avg_novehicle_noaccess = sum(no_access["NoVehicle"]) / sum_hh_noaccess
avg_poverty_noaccess = sum(no_access["USDA_Data_OHU2010"]*no_access["USDA_Data_PovertyRate"]/sum_hh_noaccess)

In [67]:
print("CENSUS TRACTS WITH NO ACCESS")
print("Average % Senior Population:", avg_seniors_noaccess)
print("Average % Black Population:", avg_black_noaccess)
print("Average % Disabled Population:", avg_disability_noaccess)

print("Average % Households that are SNAP Recipients:", avg_SNAP_noaccess)
print("Average % Single Head Households:", avg_single_noaccess)
print("Average % Household with No Vehicle:", avg_novehicle_noaccess)
print("Average Poverty Rate:", avg_poverty_noaccess, "\n")

CENSUS TRACTS WITH NO ACCESS
Average % Senior Population: 0.1655660822762897
Average % Black Population: 0.18642787447672407
Average % Disabled Population: 0.1452330741871518
Average % Households that are SNAP Recipients: 0.14445625221395678
Average % Single Head Households: 0.09970952886999646
Average % Household with No Vehicle: 0.1675522493800921
Average Poverty Rate: 0.18067285866099883 



## Baseline Model 

In [97]:
chosen_store ='106749-C'          #Provided by Mihir

chosen_store_index = np.where(comm_buildings_array == chosen_store)[0][0]
chosen_store_index

491

In [98]:
#New residential buildings that will have access based on commercial building
new_building_with_access = res_comm_access_matrix_half_mile[:, 491]
new_building_with_access = new_building_with_access.nonzero()[0]
new_building_with_access = [i for i in new_building_with_access if i not in existing_access_indices]

In [99]:
# Get IDs of all residential buildings with access
res_buildings_array[new_building_with_access]

array(['104527-R', '104532-R', '104534-R', '104535-C', '104537-R',
       '104539-R', '104542-R', '104568-R', '104570-R', '104571-R',
       '104576-R', '104577-R', '104578-R', '104583-R', '104585-R',
       '104599-R', '104607-R', '104608-R', '104609-R', '104611-R',
       '104613-R', '104633-R', '104635-R', '104637-R', '104638-R',
       '104640-R', '104644-R', '104655-R', '104660-R', '104662-R',
       '104663-R', '104666-R', '104667-R', '104669-R', '104674-R',
       '104684-R', '104685-R', '104687-R', '104690-R', '104691-R',
       '104697-R', '104700-R', '104714-C', '104715-R', '104721-R',
       '104722-R', '104725-R', '104726-C', '104728-R', '104729-R',
       '104732-R', '104733-R', '104734-R', '104735-R', '104736-R',
       '104737-R', '104740-R', '104745-R', '104746-R', '104753-R',
       '104755-R', '104756-R', '104757-R', '104760-R', '104764-R',
       '104768-R', '104769-R', '104770-R', '104775-R', '104776-R',
       '104779-R', '104783-R', '104784-R', '104790-R', '104791

In [101]:
new_access_buildings = buildings_df[buildings_df.building_id.isin(res_buildings_array[new_building_with_access])]
new_access_buildings

,CLASS,class_reco,hood,geoid10,tractce10,geometry,building_id,index_right,geo_id,B01003_001E,B25010_001E
17334,R,1-Unit Residential,Arlington,420035616002,561600,"POLYGON ((-79.95944 40.40977, -79.95948 40.409...",17335-R,252,1400000US42003480300,3491.0,2.48
18294,R,1-Unit Residential,Arlington,420035616002,561600,"POLYGON ((-79.95790 40.41066, -79.95781 40.410...",18295-R,60,1400000US42003161000,2929.0,2.26
18297,R,1-Unit Residential,Arlington,420035616002,561600,"POLYGON ((-79.95810 40.41029, -79.95799 40.410...",18298-R,60,1400000US42003161000,2929.0,2.26
18298,R,1-Unit Residential,Arlington,420035616002,561600,"POLYGON ((-79.95706 40.41124, -79.95709 40.411...",18299-R,60,1400000US42003161000,2929.0,2.26
18302,R,1-Unit Residential,Arlington,420035616002,561600,"POLYGON ((-79.95807 40.41017, -79.95812 40.410...",18303-R,60,1400000US42003161000,2929.0,2.26
...,...,...,...,...,...,...,...,...,...,...,...
110803,R,1-Unit Residential,Central Oakland,420030405001,040500,"POLYGON ((-79.96022 40.43812, -79.96028 40.438...",110804-R,7,1400000US42003040500,3065.0,2.18
110806,R,1-Unit Residential,Central Oakland,420030405001,040500,"POLYGON ((-79.95851 40.43663, -79.95866 40.436...",110807-R,7,1400000US42003040500,3065.0,2.18
110810,R,1-Unit Residential,Central Oakland,420030405001,040500,"POLYGON ((-79.95939 40.43761, -79.95949 40.437...",110811-R,7,1400000US42003040500,3065.0,2.18
110829,R,1-Unit Residential,Central Oakland,420030405001,040500,"POLYGON ((-79.96039 40.43821, -79.96045 40.438...",110830-R,7,1400000US42003040500,3065.0,2.18


In [102]:
#Census Tracts with New Access to Gracery Stores
census_tracts_with_new_access = set(new_access_buildings['geo_id'])
census_tracts_with_new_access

{'1400000US42003040200',
 '1400000US42003040500',
 '1400000US42003040900',
 '1400000US42003050600',
 '1400000US42003051000',
 '1400000US42003060300',
 '1400000US42003060500',
 '1400000US42003161000',
 '1400000US42003480300'}

In [103]:
len(census_tracts_with_new_access)

9

In [104]:
original_buildings_without_access = np.where(res_access_array_half_mile==0)[0] #Buildings that dont have access
remaining_buildings_without_access = [i for i in original_buildings_without_access if i not in new_building_with_access]
no_access_buildings = buildings_df[buildings_df.building_id.isin(res_buildings_array[remaining_buildings_without_access])]

census_tracts_without_access = set(no_access_buildings['geo_id'])
census_tracts_without_access

{'1400000US42003050600',
 '1400000US42003051000',
 '1400000US42003060300',
 '1400000US42003060500',
 '1400000US42003070900',
 '1400000US42003080400',
 '1400000US42003080600',
 '1400000US42003080900',
 '1400000US42003090100',
 '1400000US42003090200',
 '1400000US42003100500',
 '1400000US42003101800',
 '1400000US42003101900',
 '1400000US42003130600',
 '1400000US42003130700',
 '1400000US42003130800',
 '1400000US42003140100',
 '1400000US42003140800',
 '1400000US42003141100',
 '1400000US42003141400',
 '1400000US42003151600',
 '1400000US42003151700',
 '1400000US42003191600',
 '1400000US42003191700',
 '1400000US42003191800',
 '1400000US42003202200',
 '1400000US42003260200',
 '1400000US42003261300',
 '1400000US42003270100',
 '1400000US42003270300',
 '1400000US42003270800',
 '1400000US42003271600',
 '1400000US42003281400',
 '1400000US42003281500',
 '1400000US42003290100',
 '1400000US42003300100',
 '1400000US42003310200',
 '1400000US42003320400',
 '1400000US42003320600',
 '1400000US42003320700',


In [105]:
len(census_tracts_without_access)

62

## Calculations Baseline

In [106]:
tracts_new_access = census_tracts_with_new_access
tracts_no_access = census_tracts_without_access

In [107]:
#Equity Calculations 
new_access = merged[merged.Allegheny_Tracts_AFFGEOID.isin(tracts_new_access)]
no_access = merged[merged.Allegheny_Tracts_AFFGEOID.isin(tracts_no_access)]
new_access

,OBJECTID,Allegheny_Tracts_FID_1,Allegheny_Tracts_STATEFP,Allegheny_Tracts_COUNTYFP,Allegheny_Tracts_TRACTCE,Allegheny_Tracts_AFFGEOID,GEOID,Allegheny_Tracts_NAME,Allegheny_Tracts_LSAD,Allegheny_Tracts_ALAND,...,NoVehicle,Disability,Race,SingleHH,PctSeniorPop,PctSNAPHH,PctSingleHH,PctNoVehicleHH,PctDisabilityPop,Black_Pop
31,33,72,42,3,50600,1400000US42003050600,42003050600,506.0,CT,805419,...,265,651.0,0.776,282.0,24.89,16.63,27.76,26.08,31.65,1596.0
39,41,92,42,3,60500,1400000US42003060500,42003060500,605.0,CT,662686,...,145,228.0,0.049,16.0,16.01,14.06,2.30,20.80,17.90,62.0
67,70,149,42,3,40500,1400000US42003040500,42003040500,405.0,CT,472314,...,436,225.0,0.037,0.0,3.33,0.69,0.00,33.54,6.09,137.0
72,75,157,42,3,40900,1400000US42003040900,42003040900,409.0,CT,912392,...,348,321.0,0.191,59.0,10.71,14.08,4.47,26.34,10.81,567.0
97,100,226,42,3,480300,1400000US42003480300,42003480300,4803.0,CT,2846639,...,27,481.0,0.042,101.0,17.80,11.20,6.98,1.87,14.08,143.0
138,141,315,42,3,51000,1400000US42003051000,42003051000,510.0,CT,645419,...,374,382.0,0.503,85.0,8.88,37.62,11.63,51.16,18.14,1059.0
139,142,316,42,3,60300,1400000US42003060300,42003060300,603.0,CT,1117947,...,301,287.0,0.129,0.0,13.03,17.29,0.00,25.51,12.26,302.0
170,173,395,42,3,40200,1400000US42003040200,42003040200,402.0,CT,604717,...,254,186.0,0.327,45.0,8.10,37.71,7.58,42.76,7.14,852.0


In [108]:
#Calculations 
#ACCESS 
sum_pop_access = sum(new_access["USDA_Data_Pop2010"]) 
sum_hh_access = sum(new_access["USDA_Data_OHU2010"]) 

#Population
avg_seniors_access = sum(new_access["Senior"])/sum_pop_access 
avg_black_access = sum(new_access["Black_Pop"])/sum_pop_access 
avg_disability_access = sum(new_access["Disability"])/sum_pop_access 

#Households
avg_SNAP_access = sum(new_access["SNAP"]) / sum_hh_access 
avg_single_access = sum(new_access["SingleHH"]) / sum_hh_access
avg_novehicle_access = sum(new_access["NoVehicle"]) / sum_hh_access
avg_poverty_access = sum(new_access["USDA_Data_OHU2010"]*new_access["USDA_Data_PovertyRate"]/sum_hh_access)


In [109]:
print("CENSUS TRACTS WITH ACCESS")
print("Average % Senior Population:", avg_seniors_access)
print("Average % Black Population:", avg_black_access)
print("Average % Disabled Population:", avg_disability_access)

print("Average % Households that are SNAP Recipients:", avg_SNAP_access)
print("Average % Single Head Households:", avg_single_access)
print("Average % Household with No Vehicle:", avg_novehicle_access)
print("Average Poverty Rate:", avg_poverty_access, "\n")

CENSUS TRACTS WITH ACCESS
Average % Senior Population: 0.12062561094819159
Average % Black Population: 0.2305962854349951
Average % Disabled Population: 0.1349462365591398
Average % Households that are SNAP Recipients: 0.16016898008449004
Average % Single Head Households: 0.07097163548581774
Average % Household with No Vehicle: 0.2595051297525649
Average Poverty Rate: 0.3614235365117683 



In [110]:
#NO ACCESS
sum_pop_noaccess = sum(no_access["USDA_Data_Pop2010"]) 
sum_hh_noaccess = sum(no_access["USDA_Data_OHU2010"]) 

#Population
avg_seniors_noaccess = sum(no_access["Senior"])/sum_pop_noaccess
avg_black_noaccess = sum(no_access["Black_Pop"])/sum_pop_noaccess 
avg_disability_noaccess = sum(no_access["Disability"])/sum_pop_noaccess

#Households
avg_SNAP_noaccess = sum(no_access["SNAP"]) / sum_hh_noaccess
avg_single_noaccess = sum(no_access["SingleHH"]) / sum_hh_noaccess
avg_novehicle_noaccess = sum(no_access["NoVehicle"]) / sum_hh_noaccess
avg_poverty_noaccess = sum(no_access["USDA_Data_OHU2010"]*no_access["USDA_Data_PovertyRate"]/sum_hh_noaccess)

In [111]:
print("CENSUS TRACTS WITH NO ACCESS")
print("Average % Senior Population:", avg_seniors_noaccess)
print("Average % Black Population:", avg_black_noaccess)
print("Average % Disabled Population:", avg_disability_noaccess)

print("Average % Households that are SNAP Recipients:", avg_SNAP_noaccess)
print("Average % Single Head Households:", avg_single_noaccess)
print("Average % Household with No Vehicle:", avg_novehicle_noaccess)
print("Average Poverty Rate:", avg_poverty_noaccess, "\n")

CENSUS TRACTS WITH NO ACCESS
Average % Senior Population: 0.16774234132226207
Average % Black Population: 0.18074479461533624
Average % Disabled Population: 0.14914763791910424
Average % Households that are SNAP Recipients: 0.1417903031163484
Average % Single Head Households: 0.09801945055725136
Average % Household with No Vehicle: 0.1612834528288493
Average Poverty Rate: 0.1565234471498544 

